In [1]:
import sys
sys.path.append("..")
from allennlp.predictors.predictor import Predictor
import json, stanza, pickle, datetime, time, re, os, random, pymongo, csv, copy,  \
allennlp_models.structured_prediction
from multiprocessing import Process, Manager, Pool
from pymongo import MongoClient
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from pylab import mpl

/root/miniconda3/envs/hald/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MongoClient(username="hald",password="mclab236")

In [3]:
now=datetime.datetime.now()

In [4]:
now_time=now.strftime('%Y_%m_01')

In [5]:
pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [6]:
db = client.hald

In [7]:
Gene_entities=list(db.Gene_raw.find({'addtime':now_time}))
RNA_entities=list(db.RNA_raw.find({'addtime':now_time}))
Carbohydrate_entities=list(db.Carbohydrate_raw.find({'addtime':now_time}))
Lipid_entities=list(db.Lipid_raw.find({'addtime':now_time}))
Peptide_entities=list(db.Peptide_raw.find({'addtime':now_time}))
Pharmaceutical_Preparations_entities=list(db.Pharmaceutical_Preparations_raw.find({'addtime':now_time}))
Protein_entities=list(db.Protein_raw.find({'addtime':now_time}))
Toxin_entities=list(db.Toxin_raw.find({'addtime':now_time}))

In [8]:
Mutation_entities=[]
Disease_entities=[]
for i in list(db.Mutation_raw.find({'addtime':now_time})):
    if i.get('Genomic View'):
        Mutation_entities.append(i)
for i in list(db.Disease_raw.find({'addtime':now_time})):
    if i.get('url'):
        Disease_entities.append(i)

In [9]:
names=locals()

In [10]:
for i in pairs:
    print(len(names[i+'_entities']))

1496
17
136
234
43
5
14
1
25
11694


In [11]:
for j in pairs:
    names[j+'_dict']={}
    for i in names[j+'_entities']:
        names[j+'_dict'][i.get('PMID')]=names[j+'_dict'].get(i.get('PMID'),[])+[i]

In [12]:
diff_relation_dict={}
for j in range(len(pairs)-1):
    for i in range(j+1,len(pairs)):
        for k in list(set(names[pairs[j]+'_dict'].keys()).intersection(set(names[pairs[i]+'_dict'].keys()))):
            for l in names[pairs[j]+'_dict'].get(k):
                for m in names[pairs[i]+'_dict'].get(k):
                    if l.get('sentence')==m.get('sentence') and l.get('entity')!=m.get('entity'):
                        diff_relation_dict[l.get('sentence')]=diff_relation_dict.get(l.get('sentence'),[])+[{'PMID':l.get('PMID'),
                                                                                                  'entity1':l.get('entity'),
                                                                                                   'entity2':m.get('entity'),
                                                                                                   'sentence':l.get('sentence'),
                                                                                                   'AB':l.get('AB'),
                                                                                                   'target1':l.get('target'),
                                                                                                    'target1_type':l.get('type'),
                                                                                                   'target2':m.get('target'),
                                                                                                    'target2_type':m.get('type'),
                                                                                                   'PMID':l.get('PMID'),
                                                                                                  }]

In [13]:
same_relation_dict={}
for j in range(len(pairs)):
    for k in names[pairs[j]+'_dict'].values():
        for l in range(len(k)-1):
            for m in range(l+1,len(k)):
                if k[l].get('sentence')==k[m].get('sentence'):
                    if k[l].get('entity')!=k[m].get('entity'):
                        same_relation_dict[k[l].get('sentence')]=same_relation_dict.get(k[l].get('sentence'),[])+[{'PMID':k[l].get('PMID'),
                                                                                                  'entity1':k[l].get('entity'),
                                                                                                   'entity2':k[m].get('entity'),
                                                                                                   'sentence':k[l].get('sentence'),
                                                                                                   'AB':k[l].get('AB'),
                                                                                                   'target1':k[l].get('target'),
                                                                                                    'target1_type':k[l].get('type'),
                                                                                                   'target2':k[m].get('target'),
                                                                                                    'target2_type':k[l].get('type'),
                                                                                                   'PMID':k[l].get('PMID'),
                                                                                                  }]

In [14]:
print(len(same_relation_dict))
print(len(diff_relation_dict))
print(len(list(set(list(same_relation_dict.keys())).intersection(set(list(diff_relation_dict.keys()))))))

2261
471
170


In [15]:
relation_dict={}
for i in same_relation_dict.keys():
    if diff_relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)+diff_relation_dict.get(i)

In [16]:
for i in same_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)
print(len(relation_dict))
for i in diff_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=diff_relation_dict.get(i)
print(len(relation_dict))

2261
2562


In [17]:
list(relation_dict.items())[-1]

('Here, we investigated the link between the CFH Y402H genotype and low-grade inflammation.',
 [{'PMID': '37940659',
   'entity1': 'rs10801555',
   'entity2': 'Inflammation',
   'sentence': 'Here, we investigated the link between the CFH Y402H genotype and low-grade inflammation.',
   'AB': 'Age-related macular degeneration (AMD) is a common cause of visual loss among the elderly. Genetic variants in the gene encoding complement factor H (CFH) have been identified as an AMD susceptibility gene, however, the mechanistic link is debated. Here, we investigated the link between the CFH Y402H genotype and low-grade inflammation. We recruited 153 healthy individuals, 84 participants with dry stages of AMD, and 148 participants with neovascular AMD. All participants were subjected to detailed retinal examination, and interview regarding comorbidities and lifestyle. Blood samples were analyzed for level of C-Reactive Protein (CRP), white blood cell differential count, and stained with fluoresc

In [18]:
relation_list=[]
for i in relation_dict.values():
    for j in i:
        relation_list.append(j)

In [19]:
colons=[]
not_colons=[]
for i in relation_list:
    if not re.search(':',i.get('sentence')[:50],flags=re.M|re.I):
        if re.search('\snot\s',i.get('sentence'),flags=re.M|re.I):
            not_colons.append(i)

In [20]:
len(not_colons)

316

In [21]:
len(relation_list)

6584

In [22]:
not_colons[-1]

{'PMID': '37884637',
 'entity1': 'Calcifediol',
 'entity2': 'Immunologic Deficiency Syndromes',
 'sentence': 'The Average Marginal Effects of the social determinants of Serum 25-hydroxyvitamin D concentrations deficiency in older adults were predicted using a probit model in which the outcome variable assumed two values (deficiency or not deficiency), taking as independent variables those reported in previous studies.',
 'AB': "Serum 25-hydroxyvitamin D concentrations deficiency is a growing health problem that affects a significant part of the world's population, with particularly negative consequences in children and older adults. Public health has prioritized healthy aging; thus, an investigation of the social determinants related to deficient and insufficient Serum 25-hydroxyvitamin D concentrations in older adults is needed to contribute to the implementation of comprehensive social programs focused on addressing those conditions adversely affecting the health of this group. This 

In [23]:
colons=[]
triplelist_filtered=[]
for i in relation_list:
    if re.search(':',i.get('sentence')[:50],flags=re.M|re.I):
        colons.append(copy.deepcopy(i))
    else:
        triplelist_filtered.append(copy.deepcopy(i))

In [24]:
print(len(relation_list))
print(len(colons))
print(len(triplelist_filtered))

6584
1402
5182


In [25]:
colons[0]

{'PMID': '37814955',
 'entity1': 'CD93',
 'entity2': 'GP1BB',
 'sentence': 'RESULTS: Of the 28 proteins identified as BEC-enriched, 4 were significantly associated with white matter lesion volumes (CDH5 [cadherin 5], CD93 [cluster of differentiation 93], ICAM2 [intracellular adhesion molecule 2], GP1BB [glycoprotein 1b platelet subunit beta]), while another approached significance (RSPO3 [R-Spondin 3]).',
 'AB': 'BACKGROUND: Proteins expressed by brain endothelial cells (BECs), the primary cell type of the blood-brain barrier, may serve as sensitive plasma biomarkers for neurological and neurovascular conditions, including cerebral small vessel disease. METHODS: Using data from the BLSA (Baltimore Longitudinal Study of Aging; n=886; 2009-2020), BEC-enriched proteins were identified among 7268 plasma proteins (measured with SomaScanv4.1) using an automated annotation algorithm that filtered endothelial cell transcripts followed by cross-referencing with BEC-specific transcripts reported

In [26]:
sentences_with_colons=[]
sentences_with_colons_set=set()
sentences_without_colons=[]
sentences_with_colons_dict={}
sentences_without_colons_dict={}
for i in colons:
    sentences_with_colons_dict[i.get('sentence')]=sentences_with_colons_dict.get(i.get('sentence'),[])+[copy.deepcopy(i)]
    if not re.search(r'(result(s)?: )|(objective(s)?: )|(method(s)?: )|(background(s)?: )|(purpose(s)?: )|(conclusion(s)?: )|(introduction(s)?: )|(participants(s)?: )|(aim(s)?: )|(importance(s)?: )|(discussion(s)?: )|(abstract(s)?: )|(recent finding(s)?: )|([A-Z]+: )|([A-Z]+\(S\): )|(([A-Z]+\s(&\s)?)*[A-Z]+: )|(([A-Z]+\/)*[A-Z]+: )',i.get('sentence')[:50],flags=re.M|re.I):
        sentences_with_colons.append(i.get('sentence'))
        sentences_with_colons_set.add(i.get('sentence'))
    else:
        sentences_without_colons.append(re.sub(r'.*(result(s)?: )|(objective(s)?: )|(method(s)?: )|(background(s)?: )|(purpose(s)?: )|(conclusion(s)?: )|(introduction(s)?: )|(participants(s)?: )|(aim(s)?: )|(importance(s)?: )|(discussion(s)?: )|(abstract(s)?: )|(recent finding(s)?: )|([A-Z]+: )|([A-Z]+\(S\): )|(([A-Z]+\s(&\s)?)*[A-Z]+: )|(([A-Z]+\/)*[A-Z]+: )','',copy.deepcopy(i).get('sentence'),flags=re.M|re.I))

In [27]:
len(sentences_without_colons)

1388

In [28]:
len(sentences_with_colons)

14

## List sentences_with_colons_set and define sentences_with_colons_filtered with normalized sentences!!!

In [29]:
# Need to redefine!!!
sentences_with_colons_filtered=[]

In [30]:
sentences_with_colons_filtered=sentences_with_colons_filtered+sentences_without_colons

In [31]:
len(sentences_with_colons_filtered)

1388

In [32]:
sentence_colons={}
for i in sentences_with_colons_filtered:
    for j,k in sentences_with_colons_dict.items():
        if i[-30:-1]==j[-30:-1]:
            for l in k:
                l.update({'sentence_transformed':i})
            sentence_colons[i]=k

In [33]:
len(sentence_colons)

592

In [34]:
len(sentences_with_colons_dict)

602

In [35]:
triplelist_filtered[0]

{'PMID': '37940659',
 'entity1': 'CRP',
 'entity2': 'CD4',
 'sentence': 'In the neovascular stage of AMD, there was no difference in proportion of CD4+ cells or CRP levels according to genotype.',
 'AB': 'Age-related macular degeneration (AMD) is a common cause of visual loss among the elderly. Genetic variants in the gene encoding complement factor H (CFH) have been identified as an AMD susceptibility gene, however, the mechanistic link is debated. Here, we investigated the link between the CFH Y402H genotype and low-grade inflammation. We recruited 153 healthy individuals, 84 participants with dry stages of AMD, and 148 participants with neovascular AMD. All participants were subjected to detailed retinal examination, and interview regarding comorbidities and lifestyle. Blood samples were analyzed for level of C-Reactive Protein (CRP), white blood cell differential count, and stained with fluorescent antibodies to differentiate CD4+ and CD8+ T cells. CFH Y402H genotyping was performe

In [36]:
all_relations=copy.deepcopy(sentence_colons)
for i in triplelist_filtered:
    i.update({'sentence_transformed':i.get('sentence')})
    all_relations[i.get('sentence')]=all_relations.get(i.get('sentence'),[])+[i]

In [37]:
len(all_relations)

2552

In [38]:
all_relations_list=list(all_relations.keys())

# Deal with all

## Prepare for OpenIE

In [39]:
os.makedirs('results/step3/openie/filelist/')

In [40]:
ind=1
for i in all_relations_list:
    f=open('results/step3/openie/filelist/sentence'+str(ind)+'.txt','w')
    f.write(i)
    f.close()
    ind+=1

In [45]:
f=open('results/step3/openie/filelist.txt','a')
for i in range(1,len(all_relations_list)+1):
    f.write('results/step3/openie/filelist/sentence'+str(i)+'.txt'+'\n')
f.close()

In [46]:
%%capture
!java -mx500g -cp ../stanza_corenlp/stanford-corenlp-4.3.2.jar:../stanza_corenlp/stanford-corenlp-4.3.2-models.jar:../stanza_corenlp/CoreNLP-to-HTML.xsl:../slf4j-api.jar:../stanza_corenlp/slf4j-simple.jar edu.stanford.nlp.naturalli.OpenIE -format reverb -filelist results/step3/openie/filelist.txt -output results/step3/openie/openie.txt

In [47]:
len(all_relations_list)

2552

## Prepare for Stanza

In [48]:
os.makedirs('results/step3/openie/files/')
k=1
for i in range(0,len(all_relations_list),len(all_relations_list)//4):
    pickle.dump(all_relations_list[i:i+len(all_relations_list)//4],open('results/step3/openie/files/openie'+str(k)+'.pkl','wb'))
    k+=1

In [49]:
os.makedirs('results/step3/rules/files')
k=1
for i in range(0,len(all_relations_list),len(all_relations_list)//4):
    f=open('results/step3/rules/files/rules'+str(k)+'.py','w')
    f.write('''
import stanza
import os
import datetime
import time
nlp = stanza.Pipeline('en', package='genia',download_method=None)
import pickle
raw=pickle.load(open('../../openie/files/openie'''+str(k)+'''.pkl','rb'))
import re 

def nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbtype,sentence2adv):
    sentence=i
    doc=nlp(sentence)
    words=[]
    for sent in doc.sentences:
        for word in sent.words:
            words.append(word.text)
    relations=[]
    verblist=[]
    verbprototype=[]
    adv=[]
    for sent in doc.sentences:
        for word in sent.words:
            relations.append([word.text,words[int(word.head)-1],int(word.id),int(word.head),word.deprel,word.xpos])
            if word.xpos in ['VB','VBD','VBG','VBN', 'VBP', 'VBZ']:
                verblist.append({word.text:word.xpos})
                verbprototype.append({word.text:word.lemma})
            if word.xpos in ['RB','RBR','RBS']:
                adv.append({word.text:word.xpos})
    sentence2words[i]=words
    sentence2relations[i]=relations
    sentence2verblist[i]=verblist
    sentence2verbprototype[i]=verbprototype
    sentence2adv[i]=adv


start=datetime.datetime.now()
print('Parent process %s.' % os.getpid())
sentence2words={}
sentence2relations={}
sentence2verblist={}
sentence2verbprototype={}
sentence2adv={}
for i in raw:
    nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbprototype,sentence2adv)
pickle.dump(sentence2words,open('sentence2words'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2relations,open('sentence2relations'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verblist,open('sentence2verblist'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verbprototype,open('sentence2verbprototype'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2adv,open('sentence2adv'''+str(k)+'''.pkl','wb'))
print('Waiting for all subprocesses done...')

end=datetime.datetime.now()
print("The running time is "+str((end-start).seconds)+"s")  
    ''')
    f.close()
    k+=1

In [50]:
import pickle
raw=pickle.load(open('results/step3/openie/files/openie'+str(1)+'.pkl','rb'))

### Run in shell 

In [111]:
# cd results/step3/rules/files
# for i in {1..5}; do echo "nohup python rules${i}.py >> rules.log &" | bash; done
# cd ../../../../

## Deal with OpenIE

In [51]:
openie_table=pd.read_table('results/step3/openie/openie.txt',header=None)

In [52]:
openie_table.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,results/step3/openie/filelist/sentence63.txt,0,incidence proportion,is,22.6 %,1,3,10,11,11,13,1.000,The incidence proportion of delirium in older ...,DT NN NN IN NN IN JJR NNS IN NN VBZ CD NN .,incidence proportion,be,22.6 %
1,results/step3/openie/filelist/sentence63.txt,0,incidence proportion,is in,older inpatients with cancer,1,3,5,6,6,10,1.000,The incidence proportion of delirium in older ...,DT NN NN IN NN IN JJR NNS IN NN VBZ CD NN .,incidence proportion,be in,older inpatient with cancer
2,results/step3/openie/filelist/sentence63.txt,0,older inpatients,is with,cancer,6,8,8,9,9,10,1.000,The incidence proportion of delirium in older ...,DT NN NN IN NN IN JJR NNS IN NN VBZ CD NN .,older inpatient,be with,cancer
3,results/step3/openie/filelist/sentence63.txt,0,delirium,proportion of is,22.6 %,4,5,2,11,11,13,1.000,The incidence proportion of delirium in older ...,DT NN NN IN NN IN JJR NNS IN NN VBZ CD NN .,delirium,proportion of be,22.6 %
4,results/step3/openie/filelist/sentence39.txt,0,NPM1 mutations,were associated with,FLT3 ITD,3,5,5,9,9,12,0.333,Correlation analysis showed NPM1 mutations wer...,NN NN VBD NN NNS VBD RB VBN IN NN HYPH NN CC N...,npm1 mutation,be associate with,flt3 itd


In [53]:
len(openie_table)

39668

In [54]:
df=openie_table.loc[:,[0,2,3,4,12]]

In [55]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'

In [58]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']
for i in rules_pairs:
    names[i]={}
    for k in ['results/step3/rules/files/'+i+str(j)+'.pkl' for j in range(1,5)]:
        names[i].update(pickle.load(open(k,'rb')))

In [70]:
finallist=[]
for line in range(len(df)):
    i=all_relations_list[int(df.iloc[line,0][38:-4])-1]
    verblist=sentence2verblist.get(i)
    verbrelation=''
    real_verb=''
    verbdict={}
    verbs_split=df.loc[line,3].split(' ')
    for verb in verblist:
        verbdict.update(verb)
    for indexx,one in enumerate(verbs_split):
        if verbdict.get(one):
            verbrelation=one
            relations=sentence2relations.get(i)
            relations_display=[relation[-2] for relation in relations]
            verbprototype=sentence2verbprototype.get(i)
            verbprototype_dict={}
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation
                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation
                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            verbs_split[indexx]=verb_prototype
        elif bedict.get(one):
            verbs_split[indexx]=bedict.get(one)
    if bedict.get(' '.join(verbs_split[:2])):
        final=' '.join(['be']+verbs_split[2:])
    else:
        final=' '.join(verbs_split)
    finallist.append(final)
df[17]=finallist

In [71]:
ind=1
for i in all_relations_list:
    names['sentence'+str(ind)]=i
    ind+=1

In [72]:
ind

2553

In [73]:
sentence689

'These results together show that moderate hPS1 expression by AAV9 is sufficient to rescue impaired gamma-secretase activity, synaptic and memory deficits, and neurodegeneration caused by Psen mutations in mouse models.'

In [74]:
relation_dict_copy=copy.deepcopy(all_relations)

In [75]:
len(relation_dict_copy)

2552

In [79]:
df.iloc[0,0].index('sent')

30

In [80]:
suc=[]
for i in range(1,len(relation_dict_copy)+1):
    if relation_dict_copy.get(names[df.iloc[i-1,0][30:-4]]):
        openie=[]
        for j in relation_dict_copy.get(names[df.iloc[i-1,0][30:-4]]):
            j1=j.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            j2=j.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(j1,df.iloc[i-1,1]) and re.search(j2,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target1'),df.iloc[i-1,-1],j.get('target2')]]
                openie.append([j.get('target1'),df.iloc[i-1,-1],j.get('target2')])
            elif re.search(j2,df.iloc[i-1,1]) and re.search(j1,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target2'),df.iloc[i-1,-1],j.get('target1')]]
                openie.append([j.get('target2'),df.iloc[i-1,-1],j.get('target1')]) 
        if not openie:
            suc.append(names[df.iloc[i-1,0][30:-4]])    

In [82]:
error=[]
for i in range(1,len(relation_dict_copy)+1):
    if not relation_dict_copy.get(names[df.iloc[i-1,0][30:-4]]):
        error.append(names[df.iloc[i-1,0][30:-4]])

In [83]:
len(suc)

2335

In [84]:
len(relation_dict_copy)

2552

In [85]:
len(error)

0

## Deal with AllenNLP

In [86]:
sys.path.append("..")
PREDICTOR = Predictor.from_path('../openie-model.2020.03.26.tar.gz')

In [87]:
def clean(sent):
    return re.sub(' +', ' ', sent.replace('\r', ' ').replace('\n', ' '))
def get_tags_allen(sent):
    sent = sent.replace('[', '(').replace(']', ')')
    res = PREDICTOR.predict(
      sentence=sent
    )
    return res
def parse_allen(ss):
    words = ss['words']
    res = []
    for v in ss['verbs']:
        tags = v['tags']
        d = {}
        for i in range(len(words)):
            t = tags[i].replace('I-', '').replace('B-', '')
            w = words[i]
            if t in d:
                d[t] += ' '+ w
            else:
                d[t] = w
        res.append(d)
    return res

In [88]:
sentence2allennlp={}
for i in relation_dict_copy.keys():
    res=get_tags_allen(i)
    ress=parse_allen(res)
    sentence2allennlp[i]=ress

In [89]:
for i,j in relation_dict_copy.items():
    for k in j:
        k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        for l in sentence2allennlp.get(i):
            if l.get('ARG1') and l.get('ARG2') and l.get('V'):
                if re.search(k1,l.get('ARG1')) and re.search(k2,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target1'), l.get('V') ,k.get('target2')]]
                elif re.search(k2,l.get('ARG1')) and re.search(k1,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target2'), l.get('V') ,k.get('target1')]]

In [90]:
for i,j in relation_dict_copy.items():
    for k in j:
        if k.get('allennlp'):
            print(k)
            break
    else:
        continue
    break

{'PMID': '37306504', 'entity1': 'Atherosclerosis', 'entity2': 'Cardiovascular Diseases', 'sentence': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults.', 'AB': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults. METHODS: Participants from Atherosclerosis Risk in Communities study visit 5 (mean age 75.4 +- 5.1 years) with IL-6 and IL-18 measurements were included (N = 5672). Cox regression models were used to assess associations of IL-6 and IL-18 with coronary heart disease (CHD), ischemic stroke, heart failure hospitalization (HF), global CVD (composite of CHD, stroke, and HF), AF, and all-cause 

In [91]:
len(sentence2adv)

2552

In [92]:
negativewords=['no','not','never','hardly','barely','scarcely','rarely','few','little','seldom','neither','nor']
negativewords_dict={}
for i in negativewords:
    negativewords_dict[i]=i

In [93]:
def work(i,j,triple,abnormal, negative_sentences,networkx_raw):
    adv=sentence2adv.get(i)
    words=sentence2words.get(i)
    sentence=i
    relations=sentence2relations.get(i)
    relations_display=[relation[-2] for relation in relations]
    verblist=sentence2verblist.get(i)
    verbprototype=sentence2verbprototype.get(i)
    verbprototype_dict={}
    for verbp in verbprototype:
        verbprototype_dict.update(verbp)
    G = nx.Graph()
    for word in words:
        G.add_node(word)
    G.add_node('root')
    for r in range(len(relations)):
        if relations[r][4]=='root':
            G.add_edge(relations[r][0], 'root',labels=relations[r][4])
        else:
            G.add_edge(relations[r][0], relations[r][1],labels=relations[r][4])
    for k in j:
        if verblist:
            k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(k1,sentence) and re.search(k2,sentence):
                worddict={}
                for word in words:
                    worddict[word]=word
                if re.search(k1,sentence).span()[0] < re.search(k2,sentence).span()[0]:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k1_word)[-1], target=re.split('[ -]',k2_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one 
                        if verbrelation :
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target1'),verb_prototype,k.get('target2')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target1'),'former_entity':k.get('entity1'),'former_entity_type':k.get('target1_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target2'),
                                                   'latter_entity':k.get('entity2'),'latter_entity_type':k.get('target2_type')})
                                    k['networkx']=[k.get('target1'),verb_prototype,k.get('target2')]
                                else:
                                    negative_sentences.append(i)
                else:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k2_word)[-1], target=re.split('[ -]',k1_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one
                        if verbrelation:
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target2'),verb_prototype,k.get('target1')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target2'),'former_entity':k.get('entity2'),'former_entity_type':k.get('target2_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target1'),
                                                   'latter_entity':k.get('entity1'),'latter_entity_type':k.get('target1_type')})
                                    k['networkx']=[k.get('target2'),verb_prototype,k.get('target1')]
                                else:
                                    negative_sentences.append(i)
        networkx_raw.append(k)
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    triple=manager.list()
    abnormal=manager.list()
    negative_sentences=manager.list()
    networkx_raw=manager.list()
    for i,j in relation_dict_copy.items():
        p.apply_async(work, args=(i,j,triple,abnormal,negative_sentences,networkx_raw))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 122772.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 26s


In [94]:
triple_list=list(triple)
abnormal_list=list(abnormal)
negative_list=list(negative_sentences)
networkx_raw_list=list(networkx_raw)
networkx_raw_dict={}
for i in networkx_raw_list:
    networkx_raw_dict[i.get('sentence')]=networkx_raw_dict.get(i.get('sentence'),[])+[i]

In [95]:
pickle.dump(triple_list,open('results/step3/triple_list.pkl','wb'))

In [96]:
pickle.dump(abnormal_list,open('results/step3/abnormal_list.pkl','wb'))
pickle.dump(negative_list,open('results/step3/negative_list.pkl','wb'))
pickle.dump(networkx_raw_list,open('results/step3/networkx_raw_list.pkl','wb'))
pickle.dump(networkx_raw_dict,open('results/step3/networkx_raw_dict.pkl','wb'))

In [97]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [98]:
networkx_raw_list[0]

{'PMID': '37814955',
 'entity1': 'CD93',
 'entity2': 'GP1BB',
 'sentence': 'RESULTS: Of the 28 proteins identified as BEC-enriched, 4 were significantly associated with white matter lesion volumes (CDH5 [cadherin 5], CD93 [cluster of differentiation 93], ICAM2 [intracellular adhesion molecule 2], GP1BB [glycoprotein 1b platelet subunit beta]), while another approached significance (RSPO3 [R-Spondin 3]).',
 'AB': 'BACKGROUND: Proteins expressed by brain endothelial cells (BECs), the primary cell type of the blood-brain barrier, may serve as sensitive plasma biomarkers for neurological and neurovascular conditions, including cerebral small vessel disease. METHODS: Using data from the BLSA (Baltimore Longitudinal Study of Aging; n=886; 2009-2020), BEC-enriched proteins were identified among 7268 plasma proteins (measured with SomaScanv4.1) using an automated annotation algorithm that filtered endothelial cell transcripts followed by cross-referencing with BEC-specific transcripts reported

In [99]:
len(networkx_raw_dict)

2543

In [100]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i

In [101]:
for i in networkx_raw_list:
    openie_mid=[]
    openie_final=[]
    if i.get('openie'):
        for j in i.get('openie'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                openie_mid.append(j)
        adv=sentence2adv.get(i.get('sentence_transformed'))
        adv_dict={}
        for l in adv:
            adv_dict.update(l)
        for k in openie_mid:
            for m in k[1].split(' '):
                if adv_dict.get(m):
                    break
            else:
                if k not in openie_final:
                    openie_final.append(k)
    i['openie_final']=openie_final

In [102]:
for i in networkx_raw_list:
    allennlp_final=[]
    if i.get('allennlp'):
        for j in i.get('allennlp'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                allennlp_final.append(j)
    i['allennlp_final']=allennlp_final

In [103]:
for i in networkx_raw_list:
    if i.get('networkx'):
        if bedict.get(i.get('networkx')[1]):
            i['networkx']=''
        elif i.get('networkx')[1]=='"':
            i['networkx']=''
    else:
        i['networkx']=''

In [104]:
networkx_raw_final=[]
for i in networkx_raw_list:
    if i.get('networkx') or i.get('openie_final') or i.get('allennlp_final'):
        networkx_raw_final.append(i)

In [105]:
len(networkx_raw_final)

1529

In [106]:
for i in networkx_raw_final:
    bidirection=False
    if i.get('networkx'):
        if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
            i['bidirection']=True
            continue
    if i.get('allennlp'):
        for j in i.get('openie_final'):
            if j=='associated' or j=='associate' :
                bidirection=True
                break
    if i.get('openie_final'):
        for j in i.get('openie_final'):
            for k in j[1].split(' '):
                if k=='associated' or  k=='associate' :
                    bidirection=True
                    break
            else:
                continue
            break
    i['bidirection']=bidirection

In [107]:
Relation_raw=copy.deepcopy(networkx_raw_final)

In [108]:
pickle.dump(networkx_raw_final,open('results/step3/networkx_raw_final.pkl','wb'))

In [109]:
networkx_raw_final[0]

{'PMID': '37306504',
 'entity1': 'Atherosclerosis',
 'entity2': 'Cardiovascular Diseases',
 'sentence': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults.',
 'AB': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults. METHODS: Participants from Atherosclerosis Risk in Communities study visit 5 (mean age 75.4 +- 5.1 years) with IL-6 and IL-18 measurements were included (N = 5672). Cox regression models were used to assess associations of IL-6 and IL-18 with coronary heart disease (CHD), ischemic stroke, heart failure hospitalization (HF), global CVD (composite of CHD, stroke, and HF), AF, and all-ca

In [110]:
len(networkx_raw_final)

1529

In [111]:
networkx_raw_final[0]

{'PMID': '37306504',
 'entity1': 'Atherosclerosis',
 'entity2': 'Cardiovascular Diseases',
 'sentence': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults.',
 'AB': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults. METHODS: Participants from Atherosclerosis Risk in Communities study visit 5 (mean age 75.4 +- 5.1 years) with IL-6 and IL-18 measurements were included (N = 5672). Cox regression models were used to assess associations of IL-6 and IL-18 with coronary heart disease (CHD), ischemic stroke, heart failure hospitalization (HF), global CVD (composite of CHD, stroke, and HF), AF, and all-ca

In [112]:
collections_Relation_raw=db.Relation_raw

In [113]:
now_time

'2023_12_01'

In [114]:
for i in Relation_raw:
    i['addtime']=now_time

In [115]:
collections_Relation_raw.insert_many(Relation_raw)

In [116]:
Relation_raw_all_with_disease=copy.deepcopy(Relation_raw)

In [117]:
len(Relation_raw_all_with_disease)

1529

In [118]:
Relation_raw_all_with_disease[10]

{'PMID': '37306504',
 'entity1': 'IL18',
 'entity2': 'Cardiovascular Diseases',
 'sentence': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults.',
 'AB': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults. METHODS: Participants from Atherosclerosis Risk in Communities study visit 5 (mean age 75.4 +- 5.1 years) with IL-6 and IL-18 measurements were included (N = 5672). Cox regression models were used to assess associations of IL-6 and IL-18 with coronary heart disease (CHD), ischemic stroke, heart failure hospitalization (HF), global CVD (composite of CHD, stroke, and HF), AF, and all-cause death. 

In [119]:
all_es=json.load(open('results/step2/all_es_lack.json'))

In [120]:
multi_chemical_type2={}
for i in all_es:
    if not multi_chemical_type2.get(i.get('entity')):
        multi_chemical_type2[i.get('entity')]=[i.get('type')]
    else:
        if i.get('type') not in multi_chemical_type2.get(i.get('entity')):
            multi_chemical_type2[i.get('entity')]+=[i.get('type')]

In [121]:
cxz=0
for i,j in multi_chemical_type2.items():
    if len(j)>1:
        cxz+=1
        print({i:j})
print(cxz)

{'Lipopolysaccharides': ['Carbohydrate', 'Lipid', 'Toxin']}
{'Vancomycin': ['Carbohydrate', 'Peptide']}
{'Ceramides': ['Carbohydrate', 'Lipid']}
{'Sphingomyelins': ['Carbohydrate', 'Lipid']}
{'Bleomycin': ['Carbohydrate', 'Peptide']}
{'Teicoplanin': ['Carbohydrate', 'Peptide']}
{'Lipid A': ['Carbohydrate', 'Lipid', 'Toxin']}
{'G(M1) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Gangliosides': ['Carbohydrate', 'Lipid']}
{'Glycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Sulfoglycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Glycolipids': ['Carbohydrate', 'Lipid']}
{'G(M3) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Glycopeptides': ['Carbohydrate', 'Peptide']}
{'Glycosylphosphatidylinositols': ['Carbohydrate', 'Lipid']}
{'Globosides': ['Carbohydrate', 'Lipid']}
{'Ristocetin': ['Carbohydrate', 'Peptide']}
{'Glucosylceramides': ['Carbohydrate', 'Lipid']}
{'Holothurin': ['Carbohydrate', 'Toxin']}
{'Glycerylphosphorylcholine': ['Carbohydrate', 'Lipid']}
{'Glycerophosphates': ['Carbohydrate', 'L

In [122]:
Relation_raw_all_final=[]
for i in Relation_raw_all_with_disease:
    if multi_chemical_type2.get(i.get('entity1')) and multi_chemical_type2.get(i.get('entity2')):
        for j in multi_chemical_type2.get(i.get('entity1')):
            for k in multi_chemical_type2.get(i.get('entity2')):
                mid=copy.deepcopy(i)
                mid.update({'target1_type':j,'target2_type':k})
                Relation_raw_all_final.append(mid)
    else:
        Relation_raw_all_final.append(i)
        if multi_chemical_type2.get(i.get('entity1')):
            for j in multi_chemical_type2.get(i.get('entity1')):
                if i.get('target1_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target1_type':j})
                    Relation_raw_all_final.append(mid)
        elif multi_chemical_type2.get(i.get('entity2')):
            for j in multi_chemical_type2.get(i.get('entity2')):
                if i.get('target2_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target2_type':j})
                    Relation_raw_all_final.append(mid)

In [123]:
len(Relation_raw_all_final)

1539

In [124]:
direction=set()
direction_list=[]
direction_type=[]
for i in Relation_raw_all_final:
    if i.get('openie_final'):
        if i.get('openie_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('networkx'):
        if i.get('networkx')[0]!= i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('allennlp_final'):
        if i.get('allennlp_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    if i.get('bidirection'):
        if (i.get('entity2'),(i.get('entity1'))) not in direction :
            direction.add((i.get('entity2'),(i.get('entity1'))))
            direction_list.append((i.get('entity2'),(i.get('entity1'))))
            direction_type.append((i.get('target2_type'),(i.get('target1_type'))))
        if (i.get('entity1'),(i.get('entity2'))) not in direction:
            direction.add((i.get('entity1'),(i.get('entity2'))))
            direction_list.append((i.get('entity1'),(i.get('entity2'))))
            direction_type.append((i.get('target1_type'),(i.get('target2_type'))))
    else:
        direction.add((i.get('entity1'),(i.get('entity2'))))
        direction_list.append((i.get('entity1'),(i.get('entity2'))))
        direction_type.append((i.get('target1_type'),(i.get('target2_type'))))

In [125]:
Relation_raw_all_final[10]

{'PMID': '37306504',
 'entity1': 'IL18',
 'entity2': 'Cardiovascular Diseases',
 'sentence': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults.',
 'AB': 'AIMS: Interleukin-6 (IL-6) and interleukin-18 (IL-18), important cytokines implicated in atherosclerosis and inflammaging, were assessed for associations with global cardiovascular disease (CVD), atrial fibrillation (AF), and death in older adults. METHODS: Participants from Atherosclerosis Risk in Communities study visit 5 (mean age 75.4 +- 5.1 years) with IL-6 and IL-18 measurements were included (N = 5672). Cox regression models were used to assess associations of IL-6 and IL-18 with coronary heart disease (CHD), ischemic stroke, heart failure hospitalization (HF), global CVD (composite of CHD, stroke, and HF), AF, and all-cause death. 

In [127]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']
names=locals()
for i in rules_pairs:
    names[i]={}
    for k in ['./results/step3/rules/files/'+i+str(j)+'.pkl' for j in range(1,5)]:
        names[i].update(pickle.load(open(k,'rb')))

In [128]:
Relation_raw_all_final_copy=copy.deepcopy(Relation_raw_all_final)

In [129]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i
countt=0
counttt=0
fdas=[]
for i in Relation_raw_all_final_copy:
    del_index=set()
    if i.get('sentence_transformed'):
        verblist=sentence2verblist.get(i.get('sentence_transformed'))
    else:
        verblist=sentence2verblist.get(i.get('sentence'))
    verbrelation=''
    real_verb=''
    verbdict={}
    for verb in verblist:
        verbdict.update(verb)
    for indexx,j in enumerate(i.get('openie_final')):
        one=j[1]
        negative=False
        if i.get('sentence_transformed'):
            relations=sentence2relations.get(i.get('sentence_transformed'))
            verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
        else:
            relations=sentence2relations.get(i.get('sentence'))
            verbprototype=sentence2verbprototype.get(i.get('sentence'))
        for p in one.split(' '):
            for relation in relations:
                if relation[1]==p and negativewords_dict.get(relation[0]):
                    negative=True
            if verbdict.get(p):
                verbrelation=p
                relations_display=[relation[-2] for relation in relations]
                verbprototype_dict={}
                for verbp in verbprototype:
                    verbprototype_dict.update(verbp)
                for index_passive,passive in enumerate(relations_display):
                    if passive=='aux:pass':
                        if relations[index_passive][1]==verbrelation:
                            verb_prototype=verbrelation
                            break
                    elif passive=='acl' and relations[index_passive][-1]=='VBN':
                        if relations[index_passive][0]==verbrelation:
                            verb_prototype=verbrelation
                            break
                else:
                    for v in verbprototype:
                        if v.get(verbrelation):
                            verb_prototype=v.get(verbrelation)
                            break
                if verb_prototype!=one:
                    if countt<10:
                        countt+=1
                if not negative:
                    i['openie_final'][indexx][1]=verb_prototype
            elif bedict.get(one):
                if i.get('sentence_transformed'):
                    relations=sentence2relations.get(i.get('sentence_transformed'))
                    verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
                else:
                    relations=sentence2relations.get(i.get('sentence'))
                    verbprototype=sentence2verbprototype.get(i.get('sentence'))
                for relation in relations:
                    if relation[1]==one and negativewords_dict.get(relation[0]):
                        negative=True
                if not negative:
                    i['openie_final'][indexx][1]=bedict.get(one)
        if negative:
            fdas.append(i)
            i['openie_final'][indexx]=[]
    final=[]
    for t in i.get('openie_final'):
        if t != []:
            final.append(t)
    i['openie_final']=final

In [130]:
len(fdas)

0

In [131]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i
countt=0
counttt=0
fdas=[]
for i in Relation_raw_all_final_copy:
    if i.get('sentence_transformed'):
        verblist=sentence2verblist.get(i.get('sentence_transformed'))
    else:
        verblist=sentence2verblist.get(i.get('sentence'))
    verbrelation=''
    real_verb=''
    verbdict={}
    for verb in verblist:
        verbdict.update(verb)
    for indexx,j in enumerate(i.get('allennlp_final')):
        one=j[1]
        negative=False
        if i.get('sentence_transformed'):
            relations=sentence2relations.get(i.get('sentence_transformed'))
            verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
        else:
            relations=sentence2relations.get(i.get('sentence'))
            verbprototype=sentence2verbprototype.get(i.get('sentence'))
        for relation in relations:
            if relation[1]==one and negativewords_dict.get(relation[0]):
                negative=True
        if verbdict.get(one):
            verbrelation=one
            relations_display=[relation[-2] for relation in relations]
            verbprototype_dict={}
            for relation in relations:
                if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                    negative=True
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation
                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation
                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            if verb_prototype!=one:
                if countt<10:
                    countt+=1
            if not negative:
                i['allennlp_final'][indexx][1]=verb_prototype
        elif bedict.get(one):
            if i.get('sentence_transformed'):
                relations=sentence2relations.get(i.get('sentence_transformed'))
                verbprototype=sentence2verbprototype.get(i.get('sentence_transformed'))
            else:
                relations=sentence2relations.get(i.get('sentence'))
                verbprototype=sentence2verbprototype.get(i.get('sentence'))
            for relation in relations:
                if relation[1]==one and negativewords_dict.get(relation[0]):
                    negative=True
            if not negative:
                i['allennlp_final'][indexx][1]=bedict.get(one)
        if negative:
            fdas.append(i)
            i['allennlp_final'][indexx]=[]
    final=[]
    for t in i.get('allennlp_final'):
        if t != []:
            final.append(t)
    i['allennlp_final']=final

In [132]:
len(fdas)

3

In [133]:
len(Relation_raw_all_final_copy)

1539

In [134]:
Relation_raw_all_final_copys=[]
for i in Relation_raw_all_final_copy:
    if i.get('networkx') or i.get('openie_final') or i.get('allennlp_final'):
        Relation_raw_all_final_copys.append(i)
print(len(Relation_raw_all_final_copys))

1537


In [135]:
for i in Relation_raw_all_final_copys:
    if i.get('_id'):
        del i['_id']
json.dump(Relation_raw_all_final_copys,open('results/step3/triplelist.json','w'))

In [136]:
len(Relation_raw_all_final_copys)

1537

In [137]:
collections_Relations_final=db.Relations_final

In [138]:
for i in Relation_raw_all_final_copys:
    i['addtime']=now_time

In [139]:
collections_Relations_final.insert_many(Relation_raw_all_final_copys)

In [140]:
pickle.dump(Relation_raw_all_final_copys,open('results/step3/Relation_raw_all_final_copy.pkl','wb'))

In [141]:
pickle.dump(Relation_raw,open('results/step3/Relation_raw.pkl','wb'))